# Lemur project Indri and Pyndri tests

This notebook is to provide a set of examples for how to setup and run some basic queries using the indri engine and pyndri as an interface. 

In [3]:
# A few imports
import pyndri

These are some files that we have in our current directory

In [11]:
ls

IndriBuildIndex.conf  Indri_and_Pyndri_tests.ipynb  corpus.trectext


In [9]:
!cat ./corpus.trectext

<DOC>
<DOCNO>recipe</DOCNO>
<TEXT>
First put the potatoes in the pan. Then fill with water and boil till soft.
</TEXT>
</DOC>
<DOC>
<DOCNO>history</DOCNO>
<TEXT>
Eventually, agriculturalists in Europe found potatoes easier to grow and cultivate than other staple crops, such as wheat and oats.
</TEXT>
</DOC>
<DOC>
<DOCNO>news</DOCNO>
<TEXT>
It has been found that Jaimie Oliver was smuggling potatoes accross the Mexican border. No one knows yet what the meaning of this potato fiasco is.
</TEXT>
</DOC>


Indri is a set of tools for indexing and querying documents. <br>
Building an index involves creating a config file specifying the corpus to use, the stemmer and stop words. Here is an example:

```
<parameters>
    <index>index/</index>
    <memory>1024M</memory>
    <storeDocs>true</storeDocs>
    <corpus><path>corpus.trectext</path><class>trectext</class></corpus>
    <stemmer><name>krovetz</name></stemmer>
    <stopper>  
      <word>a</word>
      <word>about</word>
      <word>above</word>
      <word>according</word>
      <word>across</word>
      <word>after</word>
    </stopper>
</parameters>
```

In [10]:
!cat ./IndriBuildIndex.conf

<parameters>
<index>index/</index>
<memory>1024M</memory>
<storeDocs>true</storeDocs>
<corpus><path>corpus.trectext</path><class>trectext</class></corpus>
<stemmer><name>krovetz</name></stemmer>
</parameters>


In [3]:
!IndriBuildIndex IndriBuildIndex.conf

kstem_add_table_entry: Duplicate word emeritus will be ignored.
kstem_add_table_entry: Duplicate word emeritus will be ignored.
0:00: Opened repository index/
0:00: Opened corpus.trectext
0:00: Documents parsed: 3 Documents indexed: 0
0:00: Closed corpus.trectext
0:00: Closing index
0:00: Finished


In [4]:
ls

IndriBuildIndex.conf          corpus.trectext  run.params
Indri_and_Pyndri_tests.ipynb  index/


In [17]:
!cat run.params

<parameters>
  <index>index</index>
  <trecFormat>true</trecFormat>
  <count>1000</count>
  <query>
    <number>115</number>
    <text>
      #weight( 1.0 Jaimie 1.0 potatoes  1.0 the  )
    </text>
  </query>
  <query>
    <number>116</number>
    <text>
      #weight( 1.0 water 1.0 potatoes  1.0 the  )
    </text>
  </query>
  <stopper>
    <word>small</word>
    <word>the</word>
  </stopper>
</parameters>

In [18]:
!IndriRunQuery run.params

kstem_add_table_entry: Duplicate word emeritus will be ignored.
115 Q0 news 1 -3.87809 indri
115 Q0 recipe 2 -3.88378 indri
115 Q0 history 3 -3.88577 indri
116 Q0 recipe 1 -3.87574 indri
116 Q0 history 2 -3.88577 indri
116 Q0 news 3 -3.88613 indri
